## 1. Import libraries

In [37]:
# -*- coding: utf-8 -*-

!python anchor/model_test.py anchor/data/test/锗/C004-f-f.png
#import anchor.model_test

Prediction: top3: [(3541, '99.70'), (452, '0.30'), (3094, '0.00')]


2019-05-25 15:38:44.250524: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2
Using TensorFlow backend.


## 2. Set up parameters

In [8]:
random.seed(888)
np.random.seed(888)
tf.set_random_seed(888)

IMG_SIZE = 96

In [ ]:
# coding: utf-8
"""
    Test file or on a subset of 2013 CASIA competition data

"""

import os
import codecs
import random
import copy
import argparse
import numpy as np
import tensorflow as tf
from scipy.misc import imread
from keras.utils.np_utils import to_categorical
from keras import backend
from the_model import model_8

parser = argparse.ArgumentParser()
parser.add_argument("-p", "--pred", action="store_true")
parser.add_argument("infile", nargs="?", type=argparse.FileType('rb'))
args = parser.parse_args()

random.seed(888)
np.random.seed(888)
tf.set_random_seed(888)

IMG_SIZE = 96

Pred_Details = False
if args.pred or args.infile:
    Pred_Details = True

TEST_PATH = os.path.join("anchor","data", "test")
WEIGHTS_PATH = os.path.join("anchor","data", "weights08.h5")
LABELS_PATH = os.path.join("anchor","data", "labels.txt")

label_file = codecs.open(LABELS_PATH, "r", "utf-8")
klasses = [a.strip() for a in label_file.readlines()]
# print(len(klasses))
label_file.close()

model = model_8(IMG_SIZE, len(klasses))
model.load_weights(WEIGHTS_PATH)

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

if args.infile:
    test_data = np.ndarray([1, IMG_SIZE, IMG_SIZE], dtype=np.uint8)
    test_data[0] = imread(args.infile)

else:
    label_pngs = []
    for k, v in enumerate(klasses):
        for png in os.listdir(os.path.join(TEST_PATH, v)):
            label_pngs.append((k, v, png))

    print("Total number of test samples:", len(label_pngs))
    test_data = np.ndarray([len(label_pngs), IMG_SIZE, IMG_SIZE],
                           dtype=np.uint8)
    test_label = np.ndarray([len(label_pngs)], dtype=np.uint32)

    i = 0
    for label_png in label_pngs:
        fimg = open(os.path.join(TEST_PATH, label_png[1], label_png[2]), 'rb')
        test_data[i] = imread(fimg)
        test_label[i] = label_png[0]
        fimg.close()
        i += 1

    y_test = to_categorical(test_label)

x_test = test_data.reshape(test_data.shape[0],
                           test_data.shape[1],
                           test_data.shape[2],
                           1)
x_test = x_test.astype(np.float32)
x_test /= 255.0


def top_predictions(n, pred):
    tops = []
    pred_copy = copy.copy(pred)
    for j in range(n):
        i = np.argmax(pred_copy)
        tops.append((i, "%.2f" % (pred_copy[i] * 100)))
        pred_copy[i] = 0

    return tops


if args.infile:
    preds = model.predict(x_test)
    print("Prediction: top3:", top_predictions(3, preds[0]))

elif Pred_Details:
    preds = model.predict(x_test)	
    for k, v in enumerate(preds):
        p = np.argmax(v)
        if p != test_label[k]:
            print("Wrong prediction: top3:", top_predictions(3, v),
                  "label/file:", label_pngs[k][1], label_pngs[k][2])
else:
    loss, acc = model.evaluate(x_test, y_test, batch_size=64)
    print("Loss:", loss)
    print("Accuracy:", acc)

backend.clear_session()


In [38]:
## Model

In [39]:
# coding: utf-8
"""
    One of our best models
    This model achieves 97.2% top-1 accuracy on 2013 CASIA competition data,
    better than any previously published results.
"""

from keras.models import Model
from keras.layers import (
        Input,
        Flatten,
        Dense,
        ZeroPadding2D,
        Conv2D,
        Activation,
        MaxPooling2D,
        BatchNormalization)
from keras.layers.advanced_activations import LeakyReLU


def relu():
    return LeakyReLU(alpha=0.01)


def conv_unit(input_tensor, nb_filters, mp=False, dropout=None):
    """
    one conv-relu-bn unit
    """
    x = ZeroPadding2D()(input_tensor)
    x = Conv2D(nb_filters, (3, 3))(x)
    x = relu()(x)
    x = BatchNormalization(axis=3, momentum=0.66)(x)

    if mp:
        x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)

    return x


def out_block(input_tensor, nb_classes):
    """
    FC output
    """
    x = Flatten()(input_tensor)
    x = Dense(1024)(x)
    x = relu()(x)
    x = BatchNormalization(momentum=0.66)(x)
    x = Dense(256)(x)
    x = relu()(x)
    x = BatchNormalization(momentum=0.66)(x)
    x = Dense(nb_classes)(x)
    x = Activation('softmax')(x)
    return x


def model_8(img_size, num_classes):
    """
    This is actually model N2B
    5 blocks, 14 weight layers (1-2-2-3-3--3)
    """
    inputs = Input(shape=(img_size, img_size, 1))
    x = ZeroPadding2D()(inputs)
    x = Conv2D(64, (3, 3), strides=(2, 2))(x)
    x = relu()(x)
    x = BatchNormalization(momentum=0.66)(x)
    x = conv_unit(x, 128)
    x = conv_unit(x, 128, mp=True)
    x = conv_unit(x, 256)
    x = conv_unit(x, 256, mp=True)
    x = conv_unit(x, 384)
    x = conv_unit(x, 384)
    x = conv_unit(x, 384, mp=True)
    x = conv_unit(x, 512)
    x = conv_unit(x, 512)
    x = conv_unit(x, 512, mp=True)
    x = out_block(x, num_classes)

    model = Model(inputs=inputs, outputs=x)

    return model


In [79]:
n_classes_original = 3755
WEIGHTS_PATH = os.path.join("anchor","data", "weights08.h5")



model = model_8(IMG_SIZE, n_classes_original)
model.load_weights(WEIGHTS_PATH)

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.layers.pop()

In [77]:
model.layers.add(k)